1.	Написать функцию, которая для любой квадратной матрицы вычисляет абсолютную разность между суммами элементов ее диагоналей. Входные данные – двумерный массив, каждый элемент которого представлен целым числом, выход – число, абсолютная разность. Если среди чисел на диагонали алгоритму попадается отрицательное число, то выполнение должно быть прервано и возбуждено исключение с текстом, описывающим произошедшее.

In [223]:
import numpy as np


def matrix_diag_diff(arr: list) -> int:
    """
    вычисление абсолютной разности между суммами элементов диагоналей квадратичной матрицы
    
    params
    ------
    arr : list
        квадратичная матрица, двумерный массив целых чисел
        
    returns
    ------
    int : абсолютная разность
    """
    prim_diag = arr.diagonal()  # главная диагональ
    # матрица переворачивается, берется главная диагональ в обратном порядке для сохранения индексов
    sec_diag = np.fliplr(arr).diagonal()[::-1]  # побочная диагональ
    
    for i in range(len(prim_diag)):
        if prim_diag[i] < 0:
            raise Exception(f'Элемент ({i},{i}) равный {prim_diag[i]} отрицательный')
        if sec_diag[i] < 0:
            raise Exception(f'Элемент ({len(prim_diag) - i - 1}, {i}) равный {sec_diag[i]} отрицательный')
    return np.abs(np.sum(prim_diag) - np.sum(sec_diag))

In [234]:
for i in range(5):
    print('----------------------------------------------------------\nВходная матрица:')
    size = np.random.randint(2, 10)
    arr = np.random.randint(-5, 150, size=(size, size))
    print(arr)
    print('\nВывод функции:')
    print(matrix_diag_diff(arr))

----------------------------------------------------------
Входная матрица:
[[120  27  72 127  20]
 [ 51 139  77  29  88]
 [101 105 105  52  91]
 [ 16  94  58 109 118]
 [148 107 147  83  99]]

Вывод функции:
176
----------------------------------------------------------
Входная матрица:
[[ 92  77 129]
 [147  13  48]
 [ 64 108 115]]

Вывод функции:
14
----------------------------------------------------------
Входная матрица:
[[29  9]
 [ 3 34]]

Вывод функции:
51
----------------------------------------------------------
Входная матрица:
[[103  52 122  91 110 124   1  58]
 [135 132 110  66   6  25 132  86]
 [ 21  89  71 118  80  60  47 145]
 [113 100  37   6  44 131  73  20]
 [127  54 145  78  56  75 124  20]
 [149  76 147  74  77  44 101 124]
 [ 38  50   8  29  57 113  18  76]
 [101 107  82  87  86  28 129 144]]

Вывод функции:
96
----------------------------------------------------------
Входная матрица:
[[ 54  79  68  47  91 143  54 120]
 [ 83 136  46  92  81  -2  51  83]
 [140 113  

Exception: Элемент (2,2) равный -3 отрицательный

2.	а) Написать класс, который включает в себя метод, выполняющий запрос к базе данных и возвращающий результат запроса, адрес базы данных будет являться атрибутом класса
б) Класс должен реализовывать шаблон «Синглтон», т.е. иметь только один экземпляр
в) Применить к методу класса, выполняющему запрос, декоратор, который подсчитывает и выводит время исполнения метода
г) Реализовать атомарность операции запроса
д) Реализовать кеширование возвращаемых значений


In [9]:
import sqlite3  # операции с бд
from datetime import datetime
from functools import lru_cache  # кеширование


def execution_time(func):
    """
    декоратор для подсчета времени выполнения функции
    """
    def wrapper(*args, **kwargs):
        start = datetime.now()
        func_return = func(*args, **kwargs)
        finish = datetime.now()
        print(f'Время выполнения {finish - start}')
        return func_return
    return wrapper


class DBManager:
    """
    синглтон класс для выполнения подключения, запросов к базе данных
    """
    __instance__ = None  # при первом вызове у класса нет экземпляра
    db_path = r'D:\DB\database.db'  # путь к локальному расположению базы данных
    
    
    def __init__(self): 
        if DBManager.__instance__ is None: 
            DBManager.__instance__ = self 
        else: 
            raise Exception('Класс может иметь только один экземпляр') 

            
    @staticmethod        
    def get_instance(): 
        if not DBManager.__instance__: 
            DBManager() 
        return DBManager.__instance__ 

    
    @execution_time
    @lru_cache(maxsize=16)  # размер кеша maxsize указан для ускорения работы тк вызовов может быть много
    def execute_request(self, request):
        """
        выполнение запроса к базе данных
        
        params
        ------
        request : str
            текст запроса, синтаксис SQLite
        
        returns
        -------
        list of tuples
            строки из базы данных, соответсвующие запросу
        """
        try:
            con = sqlite3.connect(self.db_path)
            self.cur = con.cursor()
        except:
             raise Exception('Не удалось установить подключение к базе данных') 
        
        self.cur.execute(request)
        con.commit()
        if self.cur is None:
             raise Exception('Не удалось извлечь строки из базы данных')
        else:
            return self.cur.fetchall()

In [10]:
db = DBManager()

In [11]:
db

In [17]:
db.cur

In [12]:
sql = f'''
        select *
        from ind
        order by random()
        limit 1
      '''
rows10 = db.execute_request(sql)

Время выполнения 0:01:29.111256


In [13]:
rows10_2 = db.execute_request(sql)

Время выполнения 0:00:00
